<a href="https://colab.research.google.com/github/dreamzml/jupyter-note/blob/master/%E4%B8%AD%E5%A4%A7%E7%BD%91%E6%A0%A1%E9%A2%98%E5%BA%93%E7%88%AC%E8%99%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as nu
import pandas as pd
import re
import json

In [2]:
questionColumns = ('question','criteria','answers','explain','test_type')

In [39]:
def collectPage(url):
  strhtml = requests.get(url)
  strhtml.encoding = "utf8"

  soup=BeautifulSoup(strhtml.text)
  data = soup.select('.newsPage__content p')
  title = soup.select('title')[0].text.split('_', 1)[0]
  #return data
  questionType = 'singlechoice'

  #分隔成每题
  lists = {"singlechoice":[],"multiplechoice":[]}
  itemTemp = []
  for x in data:
    if x.text.find("知识点：") == 0 and itemTemp==[]:
      continue
    if x.text == title:
      continue

    if x.text.find('、') != -1 and x.text.find('、')<4 and x.text[0].upper() not in "ABCDEF":
      [n,t] = x.text.split('、', 1)
      if t == "单选题":
        questionType = "singlechoice"
      elif t == "多选题":
        questionType = "multiplechoice"
      elif t == "简答题":
        questionType = "subjective"
      elif t.split("(",1)[0]=="单项选择题":
        questionType = "singlechoice"
      elif t.split("(",1)[0]=="多项选择题":
        questionType = "multiplechoice"
      else:
        if itemTemp:
          lists[questionType].append(itemTemp)
        itemTemp = [x.text]
    elif x.text.find("【例题") == 0:
      if itemTemp:
        lists[questionType].append(itemTemp)
      itemTemp = [x.text]
      if x.text.find('单选】') != -1 or x.text.find('单选题】') != -1:
        questionType = "singlechoice"
      elif x.text.find('多选】') != -1 or x.text.find('多选题】') != -1:
        questionType = "multiplechoice"
    elif x.text.find("[例") == 0:
      if itemTemp:
        lists[questionType].append(itemTemp)
      itemTemp = [x.text]
      if x.text.find('单选]') != -1 or x.text.find('单选题]') != -1:
        questionType = "singlechoice"
      elif x.text.find('多选]') != -1 or x.text.find('多选题]') != -1:
        questionType = "multiplechoice"
    elif x.text.strip().find("【真题") == 0:
      if itemTemp:
        lists[questionType].append(itemTemp)
      itemTemp = [x.text]
      if x.text.find('单选】') != -1 or x.text.find('单选题】') != -1:
        questionType = "singlechoice"
      elif x.text.find('多选】') != -1 or x.text.find('多选题】') != -1:
        questionType = "multiplechoice"
    elif x.text.find('[.单选题]') != -1:
      if itemTemp:
        lists[questionType].append(itemTemp)
      itemTemp = [x.text.replace("[.单选题]","、")]
      questionType = "singlechoice"
    elif x.text.find('[.多选题]') != -1:
      if itemTemp:
        lists[questionType].append(itemTemp)
      itemTemp = [x.text.replace("[.多选题]","、")]
      questionType = "multiplechoice"
    elif x.text == "【单选题】":
      questionType = "singlechoice"
    elif x.text == "【多选题】":
      questionType = "multiplechoice"
    elif x.text == "":
      break
    else:
      itemTemp.append(x.text)
  #最后一题
  lists[questionType].append(itemTemp)
  #return lists
  def explanQuestion(i):
    if i[0].find('、') != -1:
      question = i[0].split('、',1)[1]
    elif i[0].find('【例题') != -1 and i[0].find('】') != -1:
      question = i[0].split('】',1)[1]
    elif i[0].find('[例') != -1 and i[0].find(']') != -1:
      question = i[0].split(']',1)[1]
    elif i[0].find('【真题') == 0 and i[0].find('】') != -1:
      question = i[0].split('】',1)[1]
    else:
      question = i[0]
    answers = {}
    criteria = ""
    explain = ""
    for a in i[1:]:
      if a[0].upper() in "ABCDEF" and (a[1] == '.' or a[1] == '、'):
        answers[a[0].upper()] = a[2:]
      elif a.find("【答案】") == 0:
        criteria = ",".join(a.replace("【答案】","").strip().upper().replace(",",""))
      elif a.find("[答案]") > -1:
        criteria = ",".join(a.replace("[答案]","").strip().upper().replace(",",""))
      elif a.find("答案:") > -1:
        criteria = ",".join(a.replace("答案:","").strip().upper().replace(",",""))
      elif a.find("【解析】") == 0:
        explain = a.replace("【解析】","").strip()
      elif a.find("[解析]") > -1:
        explain = a.replace("[解析]","").replace("【知识点】","").strip()
      elif a.find("解析:") > -1:
        explain = a.replace("解析:","").strip()
      elif a.find("知识点：") == 0:
        explain = a.replace("知识点：","").strip()
      else:
        if explain != "":
          explain += a.strip()
        # else:
        #   print(a.text)
        #print(a)
    return [question, answers, criteria, explain]

  questionList = []
  for i in lists["singlechoice"]:
    question = explanQuestion(i)
    question.append("singlechoice")
    questionList.append(question)
  for i in lists["multiplechoice"]:
    question = explanQuestion(i)
    question.append("multiplechoice")
    questionList.append(question)
    
  return pd.DataFrame(questionList, columns=questionColumns)

In [4]:
def getPagesList():
  url = "https://www.wangxiao.cn/jjs/zjjs_moni/"
  strhtml = requests.get(url)
  strhtml.encoding = "utf8"

  soup=BeautifulSoup(strhtml.text)
  data = soup.select('.newsList ul li a')
  return data

In [41]:
data = getPagesList()

listsData = pd.DataFrame([], columns=questionColumns)
for u in data:
  url = "https:"+u.get("href")
  print(u.text, url)
  listsDataItem = collectPage(url)
  print("成功采集数量", listsDataItem.shape[0])
  listsData = listsData.append(listsDataItem)

2021年中级经济师考试《运输经济》精选题 https://www.wangxiao.cn/jjs/2944317.html
成功采集数量 5
2021年中级经济师考试《经济基础知识》试题及答案 https://www.wangxiao.cn/jjs/2944102.html
成功采集数量 10
2021年中级经济师考试《中级经济基础》精选题及答案 https://www.wangxiao.cn/jjs/2944101.html
成功采集数量 10
2021年中级经济师《经济基础知识》章节练习 https://www.wangxiao.cn/jjs/2944100.html
成功采集数量 10
2021年中级经济师考试《经济基础》精选题及答案 https://www.wangxiao.cn/jjs/2923781.html
成功采集数量 10
2021年经济师考试《中级经济基础》预习试题 https://www.wangxiao.cn/jjs/2923762.html
成功采集数量 10
2021年中级经济师考试《中级经济基础》常考题 https://www.wangxiao.cn/jjs/2893736.html
成功采集数量 40
2021年经济师考试《中级经济基础》精选题 https://www.wangxiao.cn/jjs/2893735.html
成功采集数量 40
2021年中级经济师《经济基础》试题及答案 https://www.wangxiao.cn/jjs/2893733.html
成功采集数量 40
2021年中级经济师《中级经济基础》模拟题 https://www.wangxiao.cn/jjs/2893463.html
成功采集数量 40
2022年中级经济师考试《工商管理》章节试题 https://www.wangxiao.cn/jjs/2961636.html
成功采集数量 10
2022年中级经济师《中级工商管理专业知识和实务》试题 https://www.wangxiao.cn/jjs/2961635.html
成功采集数量 8
2022中级经济师考试《工商管理》精选题：市场营销与品牌管理 https://www.wangxiao.cn/jjs/2958785.html
成功采集数量 9
中级经济师考试《工商管理》章节练习：公司法人

In [40]:
collectPage("https://www.wangxiao.cn/jjs/2692482.html")

,question,criteria,answers,explain,test_type
0,形成旅游需求的主观条件是人们( ),"{'A': '可自由支配收入的多少', 'B': '社会地位的高低', 'C': '闲暇时间...",,,singlechoice
1,旅行社在向旅游饭店订房时，后者通常都给予前者以一定的折价优惠，这种价格属于( ),"{'A': '批零差价', 'B': '地区差价', 'C': '同业优惠', 'D': '...",,,singlechoice
2,调整产业内企业间及企业内部组织结构关系的产业政策是( )政策,"{'A': '产业结构', 'B': '产业组织', 'C': '产业技术', 'D': '...",,,singlechoice
3,根据区域旅游的历史、现状、资源和市场等规划要素的动态变化，对该区域旅游产业发展全局进行宏观战...,"{'A': '旅游资源开发规划', 'B': '旅游发展规划', 'C': '旅游区修建性详...",,,singlechoice
4,旅游设施建设项目的投资回收期指的是在每年的现金流入量与现金流出量不变的情况下，投资额与( )...,"{'A': '每年净现金流量', 'B': '每年税后利润', 'C': '每年提取的折旧费...",,,singlechoice
5,下列指标中，可用以衡量一国或地区旅游业发展水平的指标是( ),"{'A': '基本旅游收入', 'B': '非基本旅游收入', 'C': '旅游接待人天数'...",,,singlechoice
6,饭店企业因从国外进口经营所需的物资而造成的外汇支出称为( ),"{'A': '直接漏损', 'B': '间接漏损', 'C': '先期漏损', 'D': '...",,,singlechoice
7,若某地旅游收入的编辑储蓄倾向为40%，则该地的旅游乘数为( ),"{'A': '2', 'B': '2.5', 'C': '4', 'D': '5'}",,,singlechoice
8,"旅游供求在( )上的矛盾表现为旅游目的地国家出现的""旅游热点地区""和""旅游冷点地区""的不协调。","{'A': '时间', 'B': '季节', 'C': '空间', 'D': '价格'}",,,singlechoice
9,某旅游目的地政府为改善该地的可进入性而新修了一条公路，这在政府干预旅游业发展中属于( ),"{'A': '调整旅游结构', 'B': '弥补市场不足', 'C': '提供公共产品'}",,,singlechoice


In [42]:
listsData

,question,criteria,answers,explain,test_type
0,当社会成本大于私人成本时，会产生()。,"{'A': '外部经济', 'B': '外部不经济', 'C': '外部收益', 'D': ...",B,本题考查外部经济的概念。当社会收益大于私人收益时，便产生外部经济;当社会成本大于私人成本时便...,singlechoice
1,"外部性内部化的目标是资源的有效利用，即在可行的情况下,将全部()施加给造成外部性的人。","{'A': '机会成本', 'B': '社会成本', 'C': '沉没成本', 'D': '...",B,"本题考查外部性内部化的目标。外部性内部化的目标是资源的有效利用,即在可行的情况下,将全部社会...",singlechoice
2,"下列交通运输外部成本内部化的政策中,属于通过政府行政命令直接干预和控制来解决环境问题的方法是()。","{'A': '淘汰高污染燃料', 'B': '征收排污费', 'C': '对交通拥挤收费',...",A,本题考查交通运输外部成本内部化的政策选择。在交通运输外部成本内部化的政策中，属于通过政府行政...,singlechoice
3,社会环境包括()等。,"{'A': '基础设施', 'B': '文化宗教', 'C': '文物古迹', 'D': '...","A,B,C,E",本题考查社会环境的内容。社会环境包括了除自然环境以外的众多内容，如对自然条件的开发利用、基础...,multiplechoice
4,次生环境影响评价的特点有()。,"{'A': '总体属于直接影响评价类型', 'B': '总体属于间接影响评价类型', 'C'...","B,C,E",次生环境影响评价的特点(1)总体属于间接影响评价类型(2)进行定量分析的难度大(3)影响因素...,multiplechoice
...,...,...,...,...,...
0,单形选择题,{},,,multiplechoice
0,1.著作财产权中最基本的一项权能是【 】,"{'A': '技术诀窍 B、专利技术 C、货源情报', 'C': '申请专利的发明将得到临时...",,,singlechoice
1,A 2、B 3、C 4、B 5、A 6、D 7、B 8、C 9、D 10、B,{},,,singlechoice
2,D 12、C 13、B 14、D 15、D 16、A 17、C 18、A 19、C 20、B,{},,,multiplechoice


In [44]:
collectPage("https://www.wangxiao.cn/jjs/2851813.html")

,question,criteria,answers,explain,test_type
0,1.著作财产权中最基本的一项权能是【 】,"{'A': '技术诀窍 B、专利技术 C、货源情报', 'C': '申请专利的发明将得到临时...",,,singlechoice
1,A 2、B 3、C 4、B 5、A 6、D 7、B 8、C 9、D 10、B,{},,,singlechoice
2,D 12、C 13、B 14、D 15、D 16、A 17、C 18、A 19、C 20、B,{},,,multiplechoice
3,ACD 2、AB 3、ABCDE 4、ABCDE 5、ACDE,{},,,multiplechoice


In [45]:
collectPage("https://www.wangxiao.cn/jjs/2851815.html")

,question,criteria,answers,explain,test_type
0,单形选择题,{},,,multiplechoice


In [46]:
listsData[listsData['answers']==""]

,question,criteria,answers,explain,test_type
4,"某市一商场坐落在该市繁华地段,企业土地使用权证书记载占用土地的面积为6000平方米,经确定属...",{},,,singlechoice
0,单选题：,"{'A': '首次公开发行(IPO)', 'B': '股权再融资(SEO)', 'C': '...",,,singlechoice
0,多选题：,"{'A': '不得直接投资于信贷资产', 'B': '加强投资者适当性管理', 'C': '...",,,singlechoice
0,(一)单项选此题,{},,,singlechoice
0,动植物产品和其他检疫物，应当在( )实施检疫。,"{'A': '不一致的', 'B': '一致的', 'C': '相反的', 'D': '有差...",,,singlechoice
...,...,...,...,...,...
0,单形选择题,{},,,multiplechoice
0,1.著作财产权中最基本的一项权能是【 】,"{'A': '技术诀窍 B、专利技术 C、货源情报', 'C': '申请专利的发明将得到临时...",,,singlechoice
1,A 2、B 3、C 4、B 5、A 6、D 7、B 8、C 9、D 10、B,{},,,singlechoice
2,D 12、C 13、B 14、D 15、D 16、A 17、C 18、A 19、C 20、B,{},,,multiplechoice
